In [3]:
from dolfinx.io import XDMFFile
from dolfinx.mesh import meshtags_from_entities
from dolfinx.cpp.mesh import cell_entity_type
from dolfinx.io import distribute_entity_data
from dolfinx.graph import adjacencylist
from dolfinx.mesh import create_mesh
from dolfinx.cpp.mesh import to_type
from dolfinx.cpp.io import perm_gmsh
import numpy
from dolfinx.io import gmshio
from mpi4py import MPI
from dolfinx.io.gmshio import model_to_mesh
import numpy as np
import gmsh
import warnings
warnings.filterwarnings("ignore")
gmsh.initialize()

In [4]:
gmsh.model.add("test_mass")

In [5]:
cad = gmsh.model.geo
lc = 0.002


In [6]:
cad.addPoint(-0.05, 0.01, 0,lc, 1)
cad.addPoint(0.05, 0.01, 0, lc, 2)
cad.addPoint(0.05, -0.01, 0, lc, 3)
cad.addPoint(-0.05, -0.01, 0, lc, 4)
cad.addPoint(0,0,0, lc, 5)
cad.addLine(2, 3, 1)
cad.addLine(4, 1, 2)
cad.addCircleArc(1, 5, 2, 3)
cad.addCircleArc(3, 5, 4, 4)
cad.addCurveLoop([3, 1, 4, 2 ], 10)
cad.addPlaneSurface([10], 11)

11

In [7]:
vol = cad.extrude([(2, 11)], 0,0,0.03 )

In [8]:
cad.synchronize()

In [9]:
print(vol)

[(2, 33), (3, 1), (2, 20), (2, 24), (2, 28), (2, 32)]


In [10]:
gmsh.model.addPhysicalGroup(3, [vol[1][1]], 41, "volume")
gmsh.model.addPhysicalGroup(2, [vol[0][1],vol[2][1], vol[3][1], vol[4][1], vol[5][1]], 42, "surface")





42

In [11]:
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Circle)
Info    : [ 30%] Meshing curve 4 (Circle)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 50%] Meshing curve 14 (Line)
Info    : [ 50%] Meshing curve 15 (Circle)
Info    : [ 60%] Meshing curve 16 (Line)
Info    : [ 70%] Meshing curve 18 (Line)
Info    : [ 80%] Meshing curve 19 (Line)
Info    : [ 90%] Meshing curve 23 (Line)
Info    : [100%] Meshing curve 27 (Line)
Info    : Done meshing 1D (Wall 0.0017858s, CPU 0.003324s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 11 (Plane, Frontal-Delaunay)


Info    : [ 20%] Meshing surface 20 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 24 (Surface, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 28 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 32 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 33 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.456136s, CPU 0.453697s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 7780 nodes...
Info    : Done tetrahedrizing 7788 nodes (Wall 0.164416s, CPU 0.164558s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.43583s, CPU 0.401114s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 7.63349 (nodes removed 0 0)
Info    : It. 500 - 500 nodes created - worst tet radius 2.96923 (nodes removed 0 0)
Info    : It. 1000 - 1000 nodes c

In [12]:
gmsh.write('mirror2.msh')

Info    : Writing 'mirror2.msh'...
Info    : Done writing 'mirror2.msh'


In [13]:
model_rank = 0
domain, cell_tags, facet_tags = model_to_mesh(
    gmsh.model, MPI.COMM_WORLD, model_rank,3)

In [14]:
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
E = 130e9
nu = 0.27
mu = E / (2.0 * (1.0 + nu))
lambda_ = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))
rho = 2329
g = 9.8

In [15]:
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))
    

In [16]:
def clamped_boundary(x):
    return np.logical_or(np.isclose(x[0], -0.05), np.isclose(x[0], 0.05))


fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)



In [17]:
u_D = np.array([0, 0, 0], dtype=default_scalar_type)


In [18]:
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [19]:
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

In [20]:
ds = ufl.Measure("ds", domain=domain)

In [21]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)


def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, default_scalar_type((0, 0, -rho * g)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

In [22]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [23]:
pyvista.start_xvfb()

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
#actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped, show_edges=False)
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    figure_as_array = p.screenshot("deflection.png")

Widget(value="<iframe src='http://localhost:36051/index.html?ui=P_0x7f0778cb8ed0_0&reconnect=auto' style='widt…

In [24]:
with io.XDMFFile(domain.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

In [25]:
s = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s, s))

In [26]:
V_von_mises = fem.FunctionSpace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

In [27]:
warped.cell_data["VonMises"] = stresses.vector.array
warped.set_active_scalars("VonMises")
p = pyvista.Plotter()
p.add_mesh(warped)
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses.png")

Widget(value="<iframe src='http://localhost:36051/index.html?ui=P_0x7f0778da2450_1&reconnect=auto' style='widt…

In [28]:
gdim = domain.geometry.dim
Stress = fem.VectorFunctionSpace(domain, ("Discontinuous Lagrange", 0, (gdim,)), 9)
stress = fem.Function(Stress)
stress_expr = fem.Expression(sigma(uh), Stress.element.interpolation_points())

In [29]:
stress.interpolate(stress_expr)

In [30]:
stress.x.array.reshape((142439, 3, 3))[0]

array([[-1366.13995845,  -370.79592725,   -99.96917226],
       [ -370.79592725,   -98.04749195,   -27.46241712],
       [  -99.96917226,   -27.46241712,    34.47480182]])

In [31]:
stress_array = stress.x.array.reshape((142439, 3, 3))

In [32]:
def rot_mat(rot_z, rot_y, rot_x):
    a = rot_z
    b = rot_y
    y = rot_x
    mat = np.array([[np.cos(a)*np.cos(b), np.cos(a)*np.sin(b)*np.sin(y) - np.sin(a)*np.cos(y), np.cos(a)*np.sin(b)*np.cos(y) + np.sin(a)*np.sin(y)],[np.sin(a)*np.cos(b), np.sin(a)*np.sin(b)*np.sin(y) + np.cos(a)*np.cos(y), np.sin(a)*np.sin(b)*np.cos(y) - np.cos(a)*np.sin(y)], [-1*np.sin(b), np.cos(b)*np.sin(y), np.cos(b)*np.cos(y)]])
    return mat


def h_eigenvalues(mat):
    a = mat[0, 0]
    b = mat[1, 1]
    c = mat[0, 1]

    l1 = (a + b - np.sqrt(4 * c ** 2 + (a - b) ** 2)) / 2
    l2 = (a + b + np.sqrt(4 * c ** 2 + (a - b) ** 2)) / 2
    return l1, l2


def r_index(mat):
    l1, l2 = h_eigenvalues(mat)
    return (1/np.sqrt(l1)) - (1/np.sqrt(l2))


def stress(s1, s2, s3, s4, s5, s6):
    arr = np.array([s1, s2, s3, s4, s5, s6])
    return arr


def photoelastic(p11, p12, p44):
    arr = np.array([[p11, p12, p12, 0, 0, 0], [p12, p11, p12, 0,0,0], [p12, p12, p11, 0,0,0],[0,0,0,p44,0,0],[0,0,0,0,p44,0],[0,0,0,0,0,p44]])
    return arr


def elastic(PR, YM):
    arr = np.array([[1, -1*PR, -1*PR,0,0,0],[-1*PR, 1, -1*PR, 0,0,0], [-1*PR, -1*PR, 1,0,0,0],[0,0,0,1+PR,0,0],[0,0,0,0,1+PR,0],[0,0,0,0,0,1+PR]])
    arr = arr * (1/YM)
    return arr


def delta_b(PR, YM, p11, p12, p44, s1, s2, s3, s4, s5, s6):
    arr = np.dot(photoelastic(p11, p12, p44), np.dot(elastic(PR, YM), stress(s1, s2, s3, s4, s5, s6)))
    return arr


def b_0(nx, ny, nz):
    arr = np.array([1/(nx**2), 1/(ny**2), 1/(nz**2), 0,0,0])
    return arr


def b_new(PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz):
    delB = delta_b(PR, YM, p11, p12, p44, s1, s2, s3, s4, s5, s6)
    B0 = b_0(nx, ny, nz)
    B = delB + B0
    arr2 = np.array([[B[0], B[5], B[4]], [B[5], B[1], B[3]], [B[4], B[3], B[2]]])
    return arr2


def birefringence(rot_z, rot_x, rot_y, PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz):
    b = b_new(PR, YM, s1, s2, s3, s4, s5, s6, p11, p12, p44, nx, ny, nz)
    rot = rot_mat(rot_z, rot_x, rot_y)
    rotated = np.dot(rot, np.dot(b, np.linalg.inv(rot)))
    small_b = rotated[1:, 1:]
    delta_n = r_index(small_b)
    return delta_n


In [33]:
stress_array.shape[0]

142439

In [36]:
npoints = stress_array.shape[0]
dn = np.zeros(npoints)
for x in range(npoints):
    a = stress_array[x]
    dn[x] = birefringence(rot_z=0, rot_x=0, rot_y=0, PR=0.27, YM=130e9, s1=a[0,0], s2=a[1,1], s3=a[2,2],s4= a[1,2], s5=a[0,2],s6=a[0,1], p11=-0.094, p12=0.017,p44= -0.051, nx=3.48, ny=3.48, nz=3.48)


In [46]:
pyvista.global_theme.cmap = 'jet'
warped.cell_data["Birefringence"] = dn
warped.set_active_scalars("Birefringence")
p = pyvista.Plotter()
p.add_mesh(warped, clim=[0, 5e-8])
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses.png")

Widget(value="<iframe src='http://localhost:36051/index.html?ui=P_0x7f07285b4810_9&reconnect=auto' style='widt…

In [39]:
dn2 = np.zeros(npoints)
for x in range(npoints):
    a = stress_array[x]
    dn2[x] = birefringence(rot_z=0.2, rot_x=0, rot_y=0, PR=0.27, YM=130e9, s1=a[0,0], s2=a[1,1], s3=a[2,2],s4= a[1,2], s5=a[0,2],s6=a[0,1], p11=-0.094, p12=0.017,p44= -0.051, nx=3.48, ny=3.48, nz=3.48)


In [47]:
warped.cell_data["BirefringenceBS"] = dn2
warped.set_active_scalars("BirefringenceBS")
p = pyvista.Plotter()
p.add_mesh(warped, clim=[0, 5e-8])
p.show_axes()
if not pyvista.OFF_SCREEN:
    p.show()
else:
    stress_figure = p.screenshot(f"stresses_BS.png")

Widget(value="<iframe src='http://localhost:36051/index.html?ui=P_0x7f072879bd50_10&reconnect=auto' style='wid…